In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import norm, t

from sklearn.decomposition import PCA

## **Exercise 1**

1.1 Consider some continuous random variables generated from an unknown distribution stored in 'clean_data.npy'. Fit a univariate Gaussian distribution to this data and estimate the mean and variance of the Gaussian distribution using the maximum likelihood estimator. Report the estimated mean and variance for the Gaussian distribution and plot its probability density function for continuous random variables in the range $[-10, 20]$. Overlay this probability density function curve on the normalised histogram of the data.

**(5 marks)**


In [ ]:
clean_data = np.load("clean_data.npy").T

N = clean_data.size

mu, sigma = norm.fit(clean_data)

print("mu:", mu, "sigma:", sigma)

x = np.linspace(-10, 20, 50)
y = norm.pdf(x, mu, sigma)

# Plot
sns.histplot(clean_data, bins=50, stat='density')
plt.plot(x, y, 'r')

1.2 Next, consider a 'corrupted' version of the data used in the previous exercise, stored in 'corrupted_data.npy'. This new data is affected by some degree of outliers from an unknown source. Repeat the process of fitting a univariate Gaussian distribution to this new data (using MLE) and report the estimated mean and variance of the distribution. Plot its probability density function for continuous random variables in the range $[-10, 35]$. Overlay this probability density function curve on the normalised histogram of the new data (affected by outliers). Comment on how the new Gaussian distribution parameters estimated have changed relative to the previous values estimated in exercise 1.1, and why.

**(5 marks)**

In [ ]:
corrupted_data = np.load("corrupted_data.npy")

mu, sigma = norm.fit(corrupted_data)

print("mu:", mu, "sigma:", sigma)

x = np.linspace(-10, 20, 50)
y = norm.pdf(x, mu, sigma)

# Plot
sns.histplot(corrupted_data, bins=50, stat="density")
plt.plot(x, y, "r")

#### Comments
The corrupted data is mostly the same as the clean data, apart from some new values which appear on the right and appear to be non-normally distributed.
As a result $\mu$ has been skewed higher, and the variance (\$sigma$) has been increased.

For this data, the MLE approach for fitting a Gaussian distribution is no longer suitable.

1.3 Fit a distribution to the corrupted data from exercise 1.2 in a manner that is robust to the outliers present. Demonstrate this robustness by comparing the probability density functions of the robust and univariate Gaussian distribution for the corrupted data. Additionally compare the mean and variance estimated for both the clean data (from exercise 1.1) and the corrupted data (from exercise 1.2) based on the robust fit. Explain briefly, how your chosen approach to fitting a robust distribution to the corrupted data achieves robustness.

**(5 marks)**

In [ ]:
# TODO Explanation

corrupted_data = np.load("corrupted_data.npy")

_, mu, sigma = t.fit(clean_data)

print("mu:", mu, "sigma:", sigma)

t_x = np.linspace(-10, 20, 50)
t_y = norm.pdf(x, mu, sigma)

# Plot
sns.histplot(corrupted_data, bins=50, stat="density")
plt.plot(x, y, "r")
plt.plot(t_x, t_y, "g")

# **Exercise 2**

2.1 You are given a data array called "shape_array.npy" that comprises 7 samples organised as columns in the array. Each column vector is a 3D shape of a blood vessel of size $(N\times3)$ that has been reshaped into a vector of size $(N*3 \times 1)$. Perform PCA (using the scikit-learn implementation) of the data array and extract the principal components (eigenvectors), the coordinates of the shapes in the new co-ordinate space defined by the eigenvectors, and the singular values associated with each of the eigenvectors.

**(5 marks)**

In [ ]:
samples = np.load("shape_array.npy")

samples.shape

samples.T[0].reshape(-1, 3)

In [ ]:
fig = plt.figure(figsize=(15, 10))

# Plot each vessel (assuming 'data' contains 7 vessels)
for i in range(7):
    vessel_data = samples.T[i].reshape(-1, 3)
    x, y, z = vessel_data[:, 0], vessel_data[:, 1], vessel_data[:, 2]
    
    ax = fig.add_subplot(2, 4, i + 1, projection='3d')
    ax.scatter(x, y, z, s=1, c='r')  # Plot the 3D coordinates of the vessel

    ax.set_title(f'Vessel {i+1}')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')

In [ ]:
pca = PCA(n_components=3)
pca.fit(samples.T)

# Eigenvectors
pca.components_

In [ ]:
# Eigenvalues
pca.explained_variance_

In [ ]:
# Samples in new coordinate space compressed to 3 dimensions
pca_array = pca.transform(samples.T)

pca_array

2.2 Next, perform eigendecomposition of the covariance matrix estimated from the given data array. Compare the obtained eigenvalues with the singular values estimated from PCA in the previous step. Report any differences you might find between the two and briefly explain the reason for any differences. Find the new coordinates of each shape (i.e. column in the data array) in the new coordinate space defined by the estimated eigenvectors.

**(5 marks)**

In [ ]:
M = np.mean(samples.T, axis=0)

# Center the samples around 0
samples_standardised = (samples.T - M).T
cov = np.cov(samples_standardised)

values, vectors = np.linalg.eig(cov)

components = vectors[:,:3].real

In [ ]:
samples_standardised

In [ ]:
eig_array = np.matmul(samples_standardised.T, components)

eig_array

In [ ]:
pca_array

**Comment:** There appears to be no differences in coordinates other than the sign

2.3 Reconstruct any one shape from the provided data array using (a) new coordinates estimated from PCA in 2.1 and (b) the new coordinates estimated using eigendecomposition in 2.2. Reshape the resulting vectors from (a) and (b) into a 3D set of points of size $(N\times3)$ that represent reconstructions of the original shape. Overlay the two resulting shapes and briefly comment on their similarity. Finally, in a couple of sentences explain why PCA is often described as an approach for dimensionality reduction/data compression.

**(5 marks)**

In [ ]:
# Perform the inverse of the previous step and uncenter the data using the mean matrix
reconstructed = np.matmul(eig_array, np.linalg.pinv(components)) + M

In [ ]:
fig = plt.figure()

ax = fig.add_subplot(projection='3d')

reconstructed_vessel = reconstructed[0].T.reshape(-1, 3)
print(reconstructed_vessel)
x, y, z = reconstructed_vessel[:, 0], reconstructed_vessel[:, 1], reconstructed_vessel[:, 2]
ax.scatter(x, y, z, s=1, c='r')  # Plot the 3D coordinates of the vessel

reconstructed_vessel = pca.inverse_transform(pca_array)[0].T.reshape(-1, 3)
print(reconstructed_vessel)
x, y, z = reconstructed_vessel[:, 0], reconstructed_vessel[:, 1], reconstructed_vessel[:, 2]
ax.scatter(x, y, z, s=2, c='b')  # Plot the 3D coordinates of the vessel

**Comment:** The reconstructed vessels

# **Exercise 3: Predict Cancer Mortality Rates in US Counties**

The provided dataset comprises data collected from multiple counties in the US. The regression task for this assessment is to predict cancer mortality rates in "unseen" US counties, given some training data. The training data ('Training_data.csv') comprises various features/predictors related to socio-economic characteristics, amongst other types of information for specific counties in the country. The corresponding target variables for the training set are provided in a separate CSV file ('Training_data_targets.csv'). Use the notebooks provided for lab sessions throughout this module to provide solutions to the exercises listed below. Throughout all exercises text describing your code and answering any questions included in the exercise descriptions should be included as part of your submitted solution.


The list of predictors/features available in this data set are described below:

**Data Dictionary**

avgAnnCount: Mean number of reported cases of cancer diagnosed annually

avgDeathsPerYear: Mean number of reported mortalities due to cancer

incidenceRate: Mean per capita (100,000) cancer diagoses

medianIncome: Median income per county

popEst2015: Population of county

povertyPercent: Percent of populace in poverty

MedianAge: Median age of county residents

MedianAgeMale: Median age of male county residents

MedianAgeFemale: Median age of female county residents

AvgHouseholdSize: Mean household size of county

PercentMarried: Percent of county residents who are married

PctNoHS18_24: Percent of county residents ages 18-24 highest education attained: less than high school

PctHS18_24: Percent of county residents ages 18-24 highest education attained: high school diploma

PctSomeCol18_24: Percent of county residents ages 18-24 highest education attained: some college

PctBachDeg18_24: Percent of county residents ages 18-24 highest education attained: bachelor's degree

PctHS25_Over: Percent of county residents ages 25 and over highest education attained: high school diploma

PctBachDeg25_Over: Percent of county residents ages 25 and over highest education attained: bachelor's degree

PctEmployed16_Over: Percent of county residents ages 16 and over employed

PctUnemployed16_Over: Percent of county residents ages 16 and over unemployed

PctPrivateCoverage: Percent of county residents with private health coverage

PctPrivateCoverageAlone: Percent of county residents with private health coverage alone (no public assistance)

PctEmpPrivCoverage: Percent of county residents with employee-provided private health coverage

PctPublicCoverage: Percent of county residents with government-provided health coverage

PctPubliceCoverageAlone: Percent of county residents with government-provided health coverage alone

PctWhite: Percent of county residents who identify as White

PctBlack: Percent of county residents who identify as Black

PctAsian: Percent of county residents who identify as Asian

PctOtherRace: Percent of county residents who identify in a category which is not White, Black, or Asian

PctMarriedHouseholds: Percent of married households

BirthRate: Number of live births relative to number of women in county

In [ ]:
import os
import pandas as pd

root_dir = os.getcwd() + "/"

## Define paths to the training data and targets files
training_data_path = root_dir + 'Training_data.csv'
training_targets_path = root_dir + 'Training_data_targets.csv'

**Exercise 3.1**

Read in the training data and targets files. The training data comprises features/predictors while the targets file comprises the targets (i.e. cancer mortality rates in US counties) you need to train models to predict. Plot histograms of all features to visualise their distributions and identify outliers. Do you notice any unusual values for any of the features? If so comment on these in the text accompanying your code. Compute correlations of all features with the target variable (across the data set) and sort them according the strength of correlations. Which are the top five features with strongest correlations to the targets? Plot these correlations using the scatter matrix plotting function available in pandas and comment on at least two sets of features that show visible correlations to each other.

**(5 marks)**

In [ ]:
df_features = pd.read_csv(training_data_path)
df_target = pd.read_csv(training_targets_path)

df = pd.concat([df_features, df_target], axis=1)

df.info()

In [ ]:
df.drop(['PctSomeCol18_24', 'PctPrivateCoverageAlone'], axis=1, inplace=True)

### Unusual Values

- `PctSomeCol18_24` has mostly null values, so it is excluded.
- `PctPrivateCoverageAlone` is also excluded, as it has a large number of missing values.
- `PctEmployed16_Over` has 119 missing values, which are replaced with the mean.

In [ ]:
df.hist(bins=100,figsize=(20,15))

In [ ]:
df = df[df['MedianAge'] < 100]

# Reset the split dataframes after removing the rows
df_features = df.drop(['TARGET_deathRate'], axis=1)
df_target = df[['TARGET_deathRate']]


*   There seem to be errors/outliers in the median age features (MedianAge) with values >> 100. This is clearly an error and needs to be corrected prior to fitting regression models. (1.5 marks for code above and this discussion)

In [ ]:
abs_corr = abs(df.corrwith(df['TARGET_deathRate']))

abs_corr.sort_values(ascending=False).head(n=6)

*   Top five features with strongest correlations to targets are: incidenceRate, PctBachDeg25_Over, PctPublicCoverageAlone, medIncome and povertyPercent (2 marks for this description and code above).


*   medIncome and povertyPercent are negatively correlated to each other as you would expect.
*   povertyPercent and PctBachDeg25_Over are also negatively correlated highlighting that counties with higher degrees of poverty have fewer Bachelor graduates by the age of 25. povertyPercent also shows a strong positive correlation with PctPublicCoverageAlone, indicating that poverty stricken counties are less likely to be able to afford private healthcare coverage.
*   Similarly, PctBachDeg25_Over is negatively correlated with PctPublicCoverageAlone and positively correlated with medIncome. (1.5 marks for discussion of at least two sets of features that show correlations and code above)

**Exercise 3.2**

Create an ML pipeline using scikit-learn (as demonstrated in the lab notebooks) to pre-process the training data. (5 marks)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression

pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler())
])

features_tr = pipeline.fit_transform(df_features)

# Preview results
pd.DataFrame(features_tr, columns=df_features.columns).head()

print(features_tr.shape)
print(df_target.shape)

**Exercise 3.3**

Fit linear regression models to the pre-processed data using: Ordinary least squares (OLS), Lasso and Ridge models. Choose suitable regularisation weights for Lasso and Ridge regression and include a description in text of how they were chosen. In your submitted solution make sure you set the values for the regularisation weights equal to those you identify from your experiment(s). Quantitatively compare your results from all three models and report the best performing one. Report the overall performance of the best regression model identified. Include code for all steps above. (10 marks)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_features, df_target, test_size=0.333, random_state=42)

X_train_tr = pipeline.transform(X_train)
X_test_tr = pipeline.transform(X_test)

##### OLS

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train_tr, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

def evaluate(estimator, X, y):
    pred = estimator.predict(X)
    mse = mean_squared_error(y, pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y, pred)

    print("MSE:", mse)
    print("RMSE:",rmse)
    print("r2:",r2)

In [ ]:
evaluate(lin_reg, X_test_tr, y_test)

In [ ]:
from sklearn.model_selection import cross_validate

scores = cross_validate(lin_reg, features_tr, df_target, cv=10, scoring='neg_mean_squared_error')

scores['test_score']

#### Lasso

In [ ]:
from sklearn.linear_model import LassoCV

lasso_reg = LassoCV(cv=10)
lasso_reg.fit(X_train_tr, y_train.values.ravel())

evaluate(lasso_reg, X_test_tr, y_test)

#### Ridge

In [ ]:
from sklearn.linear_model import RidgeCV

ridge_reg = RidgeCV(cv=10)
ridge_reg.fit(X_train_tr, y_train.values.ravel())

evaluate(ridge_reg, X_test_tr, y_test)

In [ ]:
from sklearn.model_selection import cross_validate

scores = cross_validate(lasso_reg, X_test_tr, y_test.values.ravel(), scoring="neg_mean_squared_error", cv=10, return_estimator=True)

print(scores['test_score'])
scores['test_score'].mean()

In [ ]:
bestIdx = np.argmin(scores['score_time'])
estimator = scores['estimator'][bestIdx]